# Analyseer de relaties in Thuis

## Lees relaties van de website en bewaar ze in CSV-bestanden

De functie extract_relaties():
1. downloadt het relatiebestand van [de relatiepagina van de Thuis fanwebsite](https://nergensbeterdanthuis.fandom.com/nl/wiki/Relaties)
1. leest de HTML-code en bewaar ze in relaties_namen.csv

In [1]:
from thuis_html_utils import extract_relaties

extract_relaties(download=False)  #Verander in True wanneer het bestand niet bestaat in de cache

## Lees personages en bewaar in CSV-bestanden

Er zijn hoofd-, neven- en gastpersonages. De extract-functies voor hoofd- en nevenpersonages werken zoals extract_relaties()

Omdat de structuur van de gastpersonages heel inconsequent is, worden gastpersonages niet automatisch gelezen van de website. De functie extract_gastpersonages bewaart de gastpersonages die nodig zijn voor de relaties in een CSV-bestand. Wanneer later zou blijken dat er nog gastpersonages ontbreken, moeten die manueel aangevuld worden in de functie extract_gastpersonages. Aangezien gastpersonages niet van de website worden gelezen, is er ook geen *download*-parameter

Op termijn kan het nodig zijn om gastpersonages toch te lezen van de website.

In [2]:
from thuis_html_utils import extract_hoofdpersonages, extract_nevenpersonages, extract_gastpersonages

extract_hoofdpersonages(download=False)
extract_nevenpersonages(download=False)
extract_gastpersonages()

### Bewaar personages in een databank

Lees de drie CSV-bestanden (hoofd-, neven- en gastpersonages) en bewaar ze in een sqlitedatabank

In [3]:
from csv import DictReader

from thuis_db_utils import init_db, bewaar_personage_lijst
from thuis_html_utils import HOOFDPERSONAGE_CSV, NEVENPERSONAGE_CSV, GASTPERSONAGE_CSV

personages = []
with open(HOOFDPERSONAGE_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        personages.append(rij)

with open(NEVENPERSONAGE_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        personages.append(rij)    

with open(GASTPERSONAGE_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        personages.append(rij)  

init_db()
bewaar_personage_lijst(personages)


## Maak een relatiebestand met unieke ID's

Het bestand *relaties_namen.csv* bevat alleen de voornamen van de personages. Er zijn personages met dezelfde voornaam. Daarom maken we een bestand *relaties_nrs.csv* waarin elke voornaam vervangen is door de unieke ID van het personage in de databank.

We gaan ervan uit dat de scenaristen van _Thuis_ geen verwarring willen veroorzaken door in één seizoen twee personages met dezelfde voornaam te hebben. Daarom kunnen we een voornaam koppelen aan een uniek personages door de voornaam en het seizoen van de relatie te gebruiken.


Deze code kan een Exception geven wanneer de combinatie van voornaam en seizoen niet bestaat in de databank. 

In [4]:
from csv import DictReader, DictWriter
from typing import _TypedDict

from thuis_db_utils import lees_personages_voornaam
from thuis_html_utils import RELATIES_NAMEN_CSV

RELATIES_NRS_CSV = 'relaties_nrs.csv'

class RelatieNrs(_TypedDict):
    seizoen:int
    persoon_nr1:int
    persoon_nr2:int

RELATIE_NRS_HEADERS = list(RelatieNrs.__annotations__.keys())

def zoek_nr(voornaam:str, seizoen:int) -> int:
    personages = lees_personages_voornaam(voornaam)
    for personage in personages:
        if seizoen in personage['seizoenen']:
            return personage['id'] -1
    print(personages)
    raise Exception(f'Personage niet gevonden {voornaam} ({seizoen})')

relatie_namen = []
with open(RELATIES_NAMEN_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        relatie_namen.append(rij)

relatie_nrs = []
for relatie in relatie_namen:
    persoon_nr1 = zoek_nr(relatie['persoon_1'], int(relatie['seizoen']))
    persoon_nr2 = zoek_nr(relatie['persoon_2'], int(relatie['seizoen']))
    relatie_nrs.append({'persoon_nr1':persoon_nr1, 'persoon_nr2':persoon_nr2, 'seizoen':relatie['seizoen']})

with open(RELATIES_NRS_CSV, mode='w', newline='', encoding='utf-8') as f:
    writer = DictWriter(f, delimiter=';', fieldnames= RELATIE_NRS_HEADERS)
    writer.writeheader()
    writer.writerows(relatie_nrs)

## Grafennotatie

Een *graaf* bestaat uit een reeks knopen (*vertices*) die verbonden zijn door lijnen (*edges*). Twee knopen zijn verbonden als er een lijn bestaat tussen de knopen. In ons geval zijn de knopen de personages en de lijnen de relaties. Een eenvoudige voorstelling van de relaties in _Thuis_ in Python kan bestaan uit een dictionary waarbij de personages de keys zijn. De value van elke key zijn de personen waarmee het personage een relatie had

### Stap1: relatiebestand lezen

In deze stap lezen we het bestand 'relaties_nrs.csv'. Wanneer het bestand al bestaat, moeten de vorige cellen niet uitgevoerd worden.

In [1]:
from csv import DictReader

RELATIES_NRS_CSV = 'relaties_nrs.csv'

relatie_list = []
with open(RELATIES_NRS_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        relatie_list.append({key:int(value) for (key,value) in rij.items()})

relatie_list[:5]

[{'seizoen': 1, 'persoon_nr1': 9, 'persoon_nr2': 68},
 {'seizoen': 1, 'persoon_nr1': 124, 'persoon_nr2': 13},
 {'seizoen': 1, 'persoon_nr1': 34, 'persoon_nr2': 102},
 {'seizoen': 1, 'persoon_nr1': 9, 'persoon_nr2': 21},
 {'seizoen': 1, 'persoon_nr1': 199, 'persoon_nr2': 32}]

### Stap2: Een dictionary met relaties

Maak een lijst met dictionaries: key = personage_nr, value= unieke set met personages waarmee het personage een relatie heeft gehad

In [6]:
from collections import defaultdict

relatie_dict = defaultdict(set)
def add_p2_to_p1(relatie_dict:dict,  p1:int, p2: int):
    if relatie_dict.get(p1) is None:
        relatie_dict[p1] = set()
    relatie_dict[p1].add(p2)

for relatie in relatie_list:
    p1 = relatie['persoon_nr1']
    p2 = relatie['persoon_nr2']
    relatie_dict[p1].add(p2)
    relatie_dict[p2].add(p1)

relatie_dict

defaultdict(set,
            {9: {21, 68, 73, 186},
             68: {9, 46, 123, 124, 141, 177},
             124: {13, 18, 68},
             13: {55, 67, 80, 108, 124, 125, 134, 146, 151},
             34: {39, 102},
             102: {16, 25, 34, 79, 107, 128, 168, 184},
             21: {9, 28, 59, 85, 96, 196},
             199: {32},
             32: {67, 84, 92, 112, 116, 138, 153, 164, 199, 201},
             166: {36},
             36: {25, 96, 126, 166},
             188: {18},
             18: {28, 44, 63, 85, 124, 128, 173, 188},
             57: {61},
             61: {57, 193},
             25: {10, 11, 36, 102, 106, 162, 172},
             39: {34, 126},
             168: {102},
             200: {52},
             52: {109, 200},
             190: {198},
             198: {190},
             84: {32, 121},
             121: {84, 96},
             106: {25},
             109: {52},
             141: {68},
             193: {61},
             53: {48},
             48: {5

### Stap3: Pesonages sorteren op aantal relaties

Maak een lijst met personages_nrs gesorteerd op basis van het aantal relaties (van groot aantal naar klein aantal)

In [22]:
gesorteerde_keys = sorted(relatie_dict, key=lambda key: len(relatie_dict[key]), reverse=True)

for key in gesorteerde_keys[:10]:
    print(key, len(relatie_dict[key]))

32 10
13 9
102 8
18 8
8 8
25 7
48 7
74 7
68 6
21 6


### Stap4: Koppel namen aan de ID's

Gebruik de ID's van de vorige lijst om in de databank de naam te zoeken van het personage. We maken eerst een dictionary op basis van de ID om efficiënt te kunnen zoeken:

**Let op**: De ID's in de databank beginnen bij 1. De ID's in het relatiebestand beginnen bij 0

In [28]:
from collections import defaultdict
from thuis_db_utils import lees_personages
from itertools import islice

personages = lees_personages()
personages_by_id = defaultdict()
for personage in personages:
    personages_by_id[personage['id']-1]=personage['voornaam'] + " " + personage['achternaam'] #nrs in bestand beginnen bij 0, in databank begint id bij 1

for personage in islice(personages_by_id.items(), 10):
    print(personage)

(0, 'Adil Bakkal')
(1, 'Britt Van Noteghem')
(2, 'Cédric Barry')
(3, 'Christine Leysen')
(4, 'Dieter Van Aert')
(5, 'Dries Van Aken')
(6, 'Eddy Van Noteghem')
(7, 'Emma Van Damme')
(8, 'Femke De Grote')
(9, 'Frank Bomans')


### Stap 5: toon de 25 personages met de meeste relaties

We kunnen de gesorteerde keys nu gebruiken om een lijst te tonen van namen en hun aantal relaties.

Het personages met de meeste relaties in _Thuis_ is Dr. Ann De Decker.

In [32]:
for key in gesorteerde_keys[:25]:
    print(personages_by_id[key], len(relatie_dict[key]))

Ann De Decker 10
Marianne Bastiaens 9
Peggy Verbeeck 8
Rosa Verbeeck 8
Femke De Grote 8
Tom De Decker 7
Eva Verbist 7
Kaat Bomans 7
Jenny Verbeeck 6
Simonne Backx 6
Luc Bomans 6
Emma Van Damme 6
Werner Van Sevenant 5
Waldek Kosinski 5
Cois Pelckmans 5
Julia Van Capelle 5
Paulien Snackaert 5
Peter Vlerick 5
Frank Bomans 4
Bianca Bomans 4
Yves Akkermans 4
Marie Van Goethem 4
Sam Bastiaens 4
Mo Fawzi 4
Tim Cremers 4


## Een alternatieve manier via numpy matrices

We stellen een _adjacency_-matrix op: de rijen en de kolommen worden gevormd door de personages. Wanneer twee personen een relatie hebben, zetten we op de kruising tussen de rij en kolom van die personages een 1. Anders staat er een 0. Aangezien een relatie symmetrisch is, zal de matrix ook symmetrisch zijn. Omdat er verschillende seizoenen zijn, zullen we een driedimensionele matrix maken: (seizoen, personage1, personage2).

Let op: personagenummers beginnen bij 0. Ze kunnen dus meteen als index voor rijen en kolommen gebruikt worden. De nummers van de seizoenen beginnen echter vanaf 1.

### Stap 1: lees de gegevens van het CSV-bestand

We lezen de gegevens van het .csv-bestand en berekenen het aantal seizoenen (eerste kolom) en de hoogste indes van de personages

In [10]:
import numpy as np

RELATIES_NRS_CSV = 'relaties_nrs.csv'

relatie_arr = np.genfromtxt(RELATIES_NRS_CSV, dtype=np.uint8, delimiter=";", skip_header=1)
aantal_seizoenen = np.max(relatie_arr[:, 0])
hoogste_personage_index = np.max(relatie_arr)
print(f"{aantal_seizoenen = }")
print(f"{hoogste_personage_index = }")

aantal_seizoenen = 29
hoogste_personage_index = 209


### Stap 2: adjacency-matrix opstellen

Aangezien de meeste elementen 0 zullen zijn, beginnen we met een zero-matrix. Let op met het verschil tussen _aantal_ (antal_seizoenen) en _index_ (hoogste_personage_index)

Nadat de matrix is gemaakt, overlopen we de relaties en zetten op de plaats van de relatie een 1. 

In [25]:
adjacency_arr = np.zeros((aantal_seizoenen, hoogste_personage_index+1, hoogste_personage_index+1),dtype=np.uint8)
for relatie in relatie_arr:
    adjacency_arr[relatie[0]-1, relatie[1], relatie[2]] = 1
    adjacency_arr[relatie[0]-1, relatie[2], relatie[1]] = 1
adjacency_arr[0,9,68]

1

### Stap 3: Relaties combineren over de seizoenen heen

Omdat we relaties willen analyseren over alle seizoenen heen, maken we een nieuwe matrix[aantal_personages, aantan_personages]. Wanneer twee personages ooit een relatie hebben gehad, staat er een 1 op de kruising tussen rij en kolom. 

In [26]:
adjacency_totaal_arr = adjacency_arr.any(0).astype(dtype=np.uint8, casting='safe', copy=False)
np.info(adjacency_totaal_arr)

class:  ndarray
shape:  (210, 210)
strides:  (210, 1)
itemsize:  1
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x198a9992440
byteorder:  little
byteswap:  False
type: uint8


### Stap 4: Aantal verschillende relaties tellen

Om het aantal relaties te weten dat een bepaald personage heeft gehad, tellen we de waarden per kolom op (over de rijen: axis=0). We hadden natuurlijk even goed de waarden per rij kunnen optellen aangezien de matrix symmetrisch is. 

In [34]:
aantal_relaties_per_personage = adjacency_totaal_arr.sum(axis=0)
aantal_relaties_per_personage

array([ 2,  2,  1,  2,  1,  1,  3,  6,  8,  4,  2,  2,  4,  9,  2,  1,  5,
        0,  8,  1,  1,  6,  2,  2,  4,  7,  1,  2,  5,  0,  2,  3, 10,  1,
        2,  1,  4,  3,  3,  2,  2,  5,  1,  1,  1,  1,  2,  1,  7,  0,  0,
        0,  2,  1,  1,  3,  2,  1,  1,  1,  1,  2,  2,  2,  1,  1,  4,  2,
        6,  3,  1,  2,  2,  5,  7,  2,  3,  3,  1,  1,  2,  2,  0,  1,  2,
        6,  0,  1,  0,  4,  3,  2,  3,  1,  1,  1,  4,  1,  0,  2,  0,  5,
        8,  0,  1,  2,  1,  3,  2,  1,  1,  4,  1,  2,  2,  1,  1,  1,  3,
        1,  1,  2,  1,  1,  3,  1,  5,  1,  4,  2,  0,  1,  0,  0,  1,  0,
        0,  2,  1,  0,  0,  1,  1,  0,  0,  0,  1,  1,  0,  1,  2,  1,  0,
        1,  0,  0,  0,  0,  1,  0,  1,  0,  2,  1,  1,  0,  1,  1,  1,  1,
        1,  0,  1,  2,  1,  1,  1,  1,  0,  0,  1,  1,  1,  2,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1], dtype=uint32)

### Stap 5: sorteren van het aantal relaties en koppelen aan personagens

We lezen de personages uit de databank en maken een index (dictionary) op ID. (let op: id in databank begint niet bij 0).

Vervolgens sorteren we de het aantal relaties per personage (omgekeerd, dus van groot naar klein). We komen exact hetzelfde resultaat uit als bij de grafe-notatie

In [35]:
from collections import defaultdict
from thuis_db_utils import lees_personages

personages = lees_personages()
personages_by_id = defaultdict()
for personage in personages:
    personages_by_id[personage['id']-1]=personage['voornaam'] + " " + personage['achternaam'] #nrs in bestand beginnen bij 0, in databank begint id bij 1

gesorteerde_keys = np.argsort(aantal_relaties_per_personage)[::-1]
for key in gesorteerde_keys[:25]:
    print(personages_by_id[key], aantal_relaties_per_personage[key])
    

Ann De Decker 10
Marianne Bastiaens 9
Rosa Verbeeck 8
Peggy Verbeeck 8
Femke De Grote 8
Kaat Bomans 7
Tom De Decker 7
Eva Verbist 7
Jenny Verbeeck 6
Luc Bomans 6
Simonne Backx 6
Emma Van Damme 6
Waldek Kosinski 5
Werner Van Sevenant 5
Paulien Snackaert 5
Peter Vlerick 5
Cois Pelckmans 5
Julia Van Capelle 5
Tim Cremers 4
Yves Akkermans 4
Bianca Bomans 4
Marie Van Goethem 4
Mo Fawzi 4
Frank Bomans 4
Sam Bastiaens 4


## Adjacency-matrix en matrix-vermenigvuldiging

We kunnen de _adjacency_-matrix beschouwen als een transformatiematrix die een personage-vector mapt naar een personage-vextor met alle gerelateerde personages. Met andere woorden: waar komen we terecht wanneer we voor een personage alle relatiepaden volgen. Wanneer we de _adjacency_-matrix met zichzelf vermendivuldigen, zien we waar we in twee stappen terechtkomen. 

Relaties zijn symmetrisch. Als personage_1 een relatie heeft met personage_2 dan heeft personage_2 ook een relatie met personage_1. Een "relatiestap zetten" lan ook inhouden dat we een stap terugzetten, naar het oorspronkelijke personage. Wanneer het oorspronkelijke personage 1 relatie heeft, is er 1 manier om bij het personage terug te geraken. Wanneer het oorspronkelijke personage 2 relaties heeft, zijn er twee manieren om bij het personage terug te geraken.

Wanneer we de _adjacency_-matrix met zichzelf vermenigvuldigen, zien we op de diagonaal het aantal relaties voor elk personage. 

In [36]:
aantal_relaties_per_personage = (adjacency_totaal_arr @ adjacency_totaal_arr).diagonal()
gesorteerde_keys = np.argsort(aantal_relaties_per_personage)[::-1]
for key in gesorteerde_keys[:25]:
    print(personages_by_id[key], aantal_relaties_per_personage[key])


Ann De Decker 10
Marianne Bastiaens 9
Rosa Verbeeck 8
Peggy Verbeeck 8
Femke De Grote 8
Kaat Bomans 7
Tom De Decker 7
Eva Verbist 7
Jenny Verbeeck 6
Luc Bomans 6
Simonne Backx 6
Emma Van Damme 6
Waldek Kosinski 5
Werner Van Sevenant 5
Paulien Snackaert 5
Peter Vlerick 5
Cois Pelckmans 5
Julia Van Capelle 5
Tim Cremers 4
Yves Akkermans 4
Bianca Bomans 4
Marie Van Goethem 4
Mo Fawzi 4
Frank Bomans 4
Sam Bastiaens 4


## Driehoeksrelaties in Thuis

Wanneer we in drie relatiestappen bij hetzelfde personage uitkomen, is er een driehoeksrelatie: p1 <-> p2 <-> p3 <-> p1. We kunnen eens kijken of er driehoeksrelaties zijn in Thuis. 

In [38]:
aantal_relaties_per_personage = (adjacency_totaal_arr @ adjacency_totaal_arr @ adjacency_totaal_arr).diagonal()
np.count_nonzero(aantal_relaties_per_personage)

0